In [1]:
import os
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

from zsldataset import ZSLDataset
from models import ContinuousMap, ContinuousMapResidual

def dist_matrix(batch1, batch2):
    delta = batch1.unsqueeze(1) - batch2.unsqueeze(0)
    
    dist_matrix = (delta * delta).mean(dim=-1)
    
    return dist_matrix


def mag(u):
    return torch.dot(u, u)


def dist(u, v):
    return torch.dot(u - v, u - v)

## Load Yahoo dataset

Should be converted into ZSL format beforehand.

In [2]:
trainset = ZSLDataset('Data/APY/train', use_predicates=False, use_irevnet=True)
testset = ZSLDataset('Data/APY/test', use_predicates=False, use_irevnet=True)

In [3]:
bsize = 128
n_epochs = 200
num_classes = trainset.classes.shape[0]

dim_semantic = trainset[0]['class_embedding'].shape[0]
dim_visual = trainset[0]['image_embedding'].shape[0]

all_class_embeddings = torch.tensor(np.array(trainset.class_embeddings)).cuda().float()
classes_enum = torch.tensor(np.array(range(num_classes), dtype=np.int64)).cuda()

In [1]:
v_to_s = ContinuousMap(dim_source=dim_visual, dim_dest=dim_semantic, width=512).cuda()
s_to_v = ContinuousMap(dim_source=dim_semantic, dim_dest=dim_visual, width=512).cuda()

# optimizer = torch.optim.Adam(list(v_to_s.parameters()) + list(s_to_v.parameters()),
#                                 lr = 1e-3,
#                                 betas=(0.9, 0.999),
#                                 weight_decay=3e-2)

optimizer = torch.optim.SGD(list(v_to_s.parameters()) + list(s_to_v.parameters()),
                                lr = 3e-4,
                                momentum=.9,
                                weight_decay=3e-2,
                                   nesterov=True)

NameError: name 'ContinuousMap' is not defined

In [5]:
alpha1 = 50 # triplet
alpha2 = 1e-3 # surjection
alpha3 = 1e-3 # l2 loss

In [6]:
mse = torch.nn.MSELoss()
triplet_loss = torch.nn.TripletMarginLoss(margin=1, p=2)
positive_part = torch.nn.ReLU()

In [7]:
trainloader = torch.utils.data.DataLoader(trainset, 
                                             batch_size=bsize, 
                                             shuffle=True, 
                                             num_workers=4, 
                                             pin_memory=True, 
                                             drop_last=True)

testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=bsize, 
                                         shuffle=True, 
                                         num_workers=4, 
                                         pin_memory=True, 
                                         drop_last=True)

In [8]:
for e in range(n_epochs):
    v_to_s = v_to_s.train()
    s_to_v = s_to_v.train()
    
    for i, sample in enumerate(trainloader):
        optimizer.zero_grad()
        
        batch_visual = sample['image_embedding'].cuda().float()
        batch_semantic = sample['class_embedding'].cuda().float()

        batch_classes = sample['class_id'].cuda()
        
        e_hat = v_to_s(s_to_v(all_class_embeddings))
        delta = (e_hat - all_class_embeddings)
        surjection_loss = (delta * delta).sum(dim=-1).mean()
        
        s_out = v_to_s(batch_visual)
        
        same_class = classes_enum.unsqueeze(0) == batch_classes.unsqueeze(1)
        same_class = same_class.detach()

        l2_dist_loss = (dist_matrix(s_out, all_class_embeddings) * same_class.float()).mean()
        
        l2_loss = (s_out * s_out).sum(dim=-1).mean()

        loss = alpha1 * l2_dist_loss + alpha2 * surjection_loss + alpha3 * l2_loss
        
        loss.backward()
        
#         c_hat = d_matrix.argmin(dim = -1)
#         print((c_hat == batch_classes).float().mean().item())
        
        optimizer.step()
        
        print(loss.item(), end=', ')
        
    if (e+1) % 50 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.7
    
    if (e+1) % 5 == 0:
        print('\n\n- Evaluation on epoch {}'.format(e))
        
        avg_accuracy = 0.
        avg_loss = 0.
        n = 0
        
        v_to_s = v_to_s.eval() 
        s_to_v = s_to_v.eval() 
        
        with torch.no_grad():
            for i, sample in enumerate(testloader):
                n += 1
                
                batch_visual = sample['image_embedding'].cuda().float()
                batch_semantic = sample['class_embedding'].cuda().float()

                batch_classes = sample['class_id'].cuda()

                s_out = v_to_s(batch_visual)

                same_class = classes_enum.unsqueeze(0) == batch_classes.unsqueeze(1)
                same_class = same_class.detach()

                d_matrix = dist_matrix(s_out, all_class_embeddings)
                
                c_hat = d_matrix.argmin(dim = -1)

                closest_negative, _ = (d_matrix + same_class.float() * 1e6).min(dim=-1)
                furthest_positive, _ = (d_matrix * (1 - same_class.float())).max(dim=-1)

                loss = positive_part(furthest_positive - closest_negative + 5)
                loss = alpha1 * furthest_positive.mean()

                avg_loss += loss.item()
                avg_accuracy += (c_hat == batch_classes).float().mean().item()

        avg_accuracy /= n
        avg_loss /= n

        print('Average acc.: {}, Average loss:{}\n\n'.format(avg_accuracy, avg_loss))


0.4915170967578888, 0.49844157695770264, 0.4957919418811798, 0.4919854402542114, 0.4862467050552368, 0.49787646532058716, 0.48720985651016235, 0.49196577072143555, 0.4912277162075043, 0.49459144473075867, 0.49889445304870605, 0.49497780203819275, 0.48615705966949463, 0.48445624113082886, 0.49102166295051575, 0.48734182119369507, 0.4881494641304016, 0.48333361744880676, 0.4875330626964569, 0.4913105368614197, 0.48321622610092163, 0.48620253801345825, 0.4877685606479645, 0.48689937591552734, 0.476678729057312, 0.4897898733615875, 0.48716309666633606, 0.481413334608078, 0.4879518449306488, 0.4816776514053345, 0.48640763759613037, 0.48394572734832764, 0.4838361442089081, 0.48460763692855835, 0.4829810559749603, 0.4840800166130066, 0.4803541898727417, 0.48158764839172363, 0.47057920694351196, 0.4840848445892334, 0.47674912214279175, 0.48887771368026733, 0.47863277792930603, 0.478762149810791, 0.48071280121803284, 0.47183606028556824, 0.47717946767807007, 0.47423338890075684, 0.4761646986007

0.36103877425193787, 0.35178127884864807, 0.3552195727825165, 0.3548869788646698, 0.36034950613975525, 0.3530515730381012, 0.36104816198349, 0.35972288250923157, 0.35968315601348877, 0.355384886264801, 0.36170950531959534, 0.3564722239971161, 0.3558572828769684, 0.35908636450767517, 0.3609420359134674, 0.3588491976261139, 0.35429948568344116, 0.3569100797176361, 0.35644519329071045, 0.35520637035369873, 0.3577401638031006, 0.35521161556243896, 0.35196080803871155, 0.351428359746933, 0.3551003336906433, 0.3570314645767212, 0.3541903495788574, 0.3483569324016571, 0.3527510166168213, 0.3572862148284912, 0.35220709443092346, 0.3485104739665985, 0.35452115535736084, 0.3542783558368683, 0.359988272190094, 0.35424160957336426, 0.3518451154232025, 0.3463098108768463, 0.3498465418815613, 0.35264670848846436, 0.35524633526802063, 0.34890103340148926, 0.3559247553348541, 0.34747225046157837, 0.3516426384449005, 0.3529510796070099, 0.34411340951919556, 0.35206979513168335, 0.34865036606788635, 0.3

0.2931578457355499, 0.2941986918449402, 0.28929588198661804, 0.29245954751968384, 0.29339367151260376, 0.29200825095176697, 0.2910558879375458, 0.29231786727905273, 0.29428890347480774, 0.29305243492126465, 0.28603237867355347, 0.2920326590538025, 0.292283833026886, 0.28720349073410034, 0.2918691039085388, 0.2898818850517273, 0.2904588282108307, 0.288088858127594, 0.28999587893486023, 0.29499542713165283, 0.29184702038764954, 0.291816383600235, 0.29277506470680237, 0.2931694984436035, 0.28488245606422424, 0.2926607131958008, 0.2868139147758484, 0.2935035228729248, 0.2905697524547577, 0.28900450468063354, 0.2887299060821533, 0.29034948348999023, 0.29491570591926575, 0.2926223874092102, 0.29163241386413574, 0.2859703600406647, 0.2821742594242096, 0.289939820766449, 0.28822362422943115, 0.28645458817481995, 0.2850208580493927, 0.2836058437824249, 0.2885497808456421, 0.29041001200675964, 0.286548912525177, 0.2867641746997833, 0.287227600812912, 0.2841838598251343, 0.291183739900589, 0.2917

0.25122666358947754, 0.25258690118789673, 0.2548067569732666, 0.2521599531173706, 0.2511816918849945, 0.24591445922851562, 0.25150376558303833, 0.25213250517845154, 0.24945981800556183, 0.25212129950523376, 0.25128182768821716, 0.2529321610927582, 0.24779337644577026, 0.2568884491920471, 0.2512436807155609, 0.25440624356269836, 0.2523612678050995, 0.25931093096733093, 0.2550913095474243, 0.24812117218971252, 0.2507515847682953, 0.24985137581825256, 0.2505793571472168, 0.2549118995666504, 0.25780197978019714, 0.25564584136009216, 0.2514263093471527, 0.2585103213787079, 0.2517186105251312, 0.24956965446472168, 0.2507646083831787, 0.2576066255569458, 0.25541576743125916, 0.24932681024074554, 0.25275835394859314, 0.24829526245594025, 0.24474383890628815, 0.25089749693870544, 0.2567232549190521, 0.24505403637886047, 0.24988172948360443, 0.24637021124362946, 0.24307161569595337, 0.25418636202812195, 0.25629863142967224, 0.2480309158563614, 0.24963189661502838, 0.250423401594162, 0.2448242902

0.2253517359495163, 0.22814762592315674, 0.2268962562084198, 0.23010452091693878, 0.22704461216926575, 0.23151132464408875, 0.22778774797916412, 0.22822847962379456, 0.2284429520368576, 0.22412586212158203, 0.2291039079427719, 0.22630317509174347, 0.23714709281921387, 0.22834622859954834, 0.23111428320407867, 0.22688515484333038, 0.22591952979564667, 0.22750017046928406, 0.2230866402387619, 0.21918204426765442, 0.22429926693439484, 0.22772926092147827, 0.22793380916118622, 0.22280924022197723, 0.22493353486061096, 0.22710679471492767, 0.23530790209770203, 0.22527484595775604, 0.2245352864265442, 0.22701424360275269, 0.2284642606973648, 0.22047409415245056, 0.22903871536254883, 0.22853446006774902, 0.22336675226688385, 0.22216369211673737, 0.23066933453083038, 0.22356921434402466, 0.22778648138046265, 0.22900183498859406, 0.2290116250514984, 0.22049693763256073, 0.22861339151859283, 0.23068243265151978, 0.2234964221715927, 0.2279115915298462, 0.22196495532989502, 0.2259484976530075, 0.2

...

0.1730232685804367, 0.16996024549007416, 0.17437432706356049, 0.1632368415594101, 0.17012649774551392, 0.16940470039844513, 0.1627974957227707, 0.16959233582019806, 0.16717864573001862, 0.1718488186597824, 0.17316193878650665, 0.17176972329616547, 0.16946807503700256, 0.16641415655612946, 0.17013512551784515, 0.1677548736333847, 0.17499622702598572, 0.17276141047477722, 0.1679536998271942, 0.17151859402656555, 0.17077386379241943, 0.16560669243335724, 0.17122703790664673, 0.16363216936588287, 0.1710658073425293, 0.1723642796278, 0.1717401146888733, 0.17151041328907013, 0.17283833026885986, 0.17126049101352692, 0.1739082634449005, 0.17228789627552032, 0.16565349698066711, 0.1738520711660385, 0.16839443147182465, 0.1683400571346283, 0.16963407397270203, 0.17813639342784882, 0.16673442721366882, 0.16620957851409912, 0.16460275650024414, 0.1756393313407898, 0.16652584075927734, 0.1692405790090561, 0.1711650937795639, 0.16791851818561554, 0.1706666797399521, 0.1744910329580307, 0.1722055524

0.17020577192306519, 0.17119204998016357, 0.16761058568954468, 0.16714075207710266, 0.17114020884037018, 0.16716428101062775, 0.16892170906066895, 0.171829953789711, 0.1717681586742401, 0.16552157700061798, 0.16707539558410645, 0.16950717568397522, 0.16712386906147003, 0.1695064753293991, 0.17348408699035645, 0.16777586936950684, 0.17029154300689697, 0.1782664954662323, 0.1642875224351883, 0.1636202484369278, 0.1728542298078537, 0.1623554825782776, 0.1708381325006485, 0.171157106757164, 0.1736382246017456, 0.1681414246559143, 0.17209884524345398, 0.17080359160900116, 0.16918668150901794, 0.1684468686580658, 0.16547146439552307, 0.17011065781116486, 0.16874678432941437, 0.16806064546108246, 0.1676001399755478, 0.16926762461662292, 0.17307765781879425, 0.17289452254772186, 0.1684093028306961, 0.17136408388614655, 0.16649633646011353, 0.1684269905090332, 0.16690106689929962, 0.16707175970077515, 0.16901373863220215, 0.1723092645406723, 0.16571420431137085, 0.16969481110572815, 0.170892581

0.1685832291841507, 0.16629518568515778, 0.1697106510400772, 0.16667094826698303, 0.16884036362171173, 0.1692608743906021, 0.1710437536239624, 0.17150311172008514, 0.1678933948278427, 0.16511808335781097, 0.17038722336292267, 0.16659575700759888, 0.16508157551288605, 0.16625013947486877, 0.1656084507703781, 0.16640891134738922, 0.17282892763614655, 0.16464164853096008, 0.16916495561599731, 0.1674347221851349, 0.16745419800281525, 0.1688700020313263, 0.17191103100776672, 0.1662350744009018, 0.16684266924858093, 0.17112624645233154, 0.16622430086135864, 0.17072021961212158, 0.1667444258928299, 0.16591192781925201, 0.17009314894676208, 0.16688060760498047, 0.17449790239334106, 0.172914057970047, 0.1680988371372223, 0.16894254088401794, 0.1687067151069641, 0.16212384402751923, 0.16570794582366943, 0.16797852516174316, 0.16784310340881348, 0.16654974222183228, 0.1744525134563446, 0.16997230052947998, 0.1640445441007614, 0.16137275099754333, 0.166845440864563, 0.17171798646450043, 0.17195616

0.16382364928722382, 0.1690044403076172, 0.16811612248420715, 0.16928815841674805, 0.16778500378131866, 0.1630990207195282, 0.16785532236099243, 0.17326749861240387, 0.16943158209323883, 0.16594882309436798, 0.16211585700511932, 0.17381136119365692, 0.166078120470047, 0.16740094125270844, 0.174429789185524, 0.1657198965549469, 0.16646139323711395, 0.16818910837173462, 0.17087188363075256, 0.16751629114151, 0.16627193987369537, 0.1655140370130539, 0.1668466031551361, 0.1680850237607956, 0.17242948710918427, 0.16968971490859985, 0.16944968700408936, 0.16719530522823334, 0.1657509058713913, 0.16588705778121948, 0.16094040870666504, 0.1700124591588974, 0.16456729173660278, 0.16741937398910522, 0.1635550558567047, 0.16848070919513702, 0.16672208905220032, 0.16816246509552002, 0.16876792907714844, 0.1714102029800415, 0.16990238428115845, 0.16760872304439545, 0.16613450646400452, 0.1626633256673813, 0.1661309003829956, 0.16431571543216705, 0.16697847843170166, 0.17063471674919128, 0.166044011

0.16279667615890503, 0.16688141226768494, 0.1676826775074005, 0.16631406545639038, 0.1660911738872528, 0.16598355770111084, 0.17184868454933167, 0.1705368161201477, 0.16723693907260895, 0.16757433116436005, 0.16403783857822418, 0.1690928339958191, 0.1726686954498291, 0.16620667278766632, 0.16959287226200104, 0.16450360417366028, 0.16515588760375977, 0.17124715447425842, 0.1672040969133377, 0.1636640578508377, 0.16547173261642456, 0.16527514159679413, 0.16954198479652405, 0.1643037497997284, 0.16881385445594788, 0.16842854022979736, 0.16464917361736298, 0.16582369804382324, 0.16773663461208344, 0.1639973223209381, 0.16655518114566803, 0.1658642739057541, 0.16634143888950348, 0.16397175192832947, 0.16466160118579865, 0.16712650656700134, 0.1737411618232727, 0.16845791041851044, 0.167279914021492, 0.16422444581985474, 0.16512872278690338, 0.16526174545288086, 0.16040991246700287, 0.16166454553604126, 0.16321994364261627, 0.16396145522594452, 0.16658876836299896, 0.16496554017066956, 0.167

# Best results

with words only: 34.6%

with words+attributes:

# With attributes

In [1]:
import os
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

from zsldataset import ZSLDataset
from models import ContinuousMap, ContinuousMapResidual

def dist_matrix(batch1, batch2):
    delta = batch1.unsqueeze(1) - batch2.unsqueeze(0)
    
    dist_matrix = (delta * delta).mean(dim=-1)
    
    return dist_matrix


def mag(u):
    return torch.dot(u, u)


def dist(u, v):
    return torch.dot(u - v, u - v)

In [2]:
from models import EncoderAttributes, DecoderAttributes

In [3]:
trainset = ZSLDataset('Data/APY/train', use_predicates=True, use_irevnet=True)
testset = ZSLDataset('Data/APY/test', use_predicates=True, use_irevnet=True)

In [4]:
bsize = 128
n_epochs = 200
num_classes = trainset.classes.shape[0]

dim_semantic = trainset[0]['class_embedding'].shape[0]
dim_visual = trainset[0]['image_embedding'].shape[0]
dim_attributes = trainset[0]['class_predicates'].shape[0]

all_class_embeddings = torch.tensor(np.array(trainset.class_embeddings)).cuda().float()
all_class_predicates = torch.tensor(np.array(trainset.class_predicates)).cuda().float()
classes_enum = torch.tensor(np.array(range(num_classes), dtype=np.int64)).cuda()

In [5]:
v_to_s = DecoderAttributes(dim_source=dim_visual, dim_target1=dim_attributes, dim_target2=dim_semantic, width=512).cuda()
s_to_v = EncoderAttributes(dim_source1=dim_semantic, dim_source2=dim_attributes, dim_target=dim_visual, width=512).cuda()

# optimizer = torch.optim.Adam(list(v_to_s.parameters()) + list(s_to_v.parameters()),
#                                 lr = 1e-3,
#                                 betas=(0.9, 0.999),
#                                 weight_decay=3e-2)

optimizer = torch.optim.SGD(list(v_to_s.parameters()) + list(s_to_v.parameters()),
                                lr = 1e-4,
                                momentum=.9,
                                weight_decay=3e-3,
                                nesterov=True)

In [6]:
mse = torch.nn.MSELoss()
triplet_loss = torch.nn.TripletMarginLoss(margin=1, p=2)
positive_part = torch.nn.ReLU()

In [7]:
trainloader = torch.utils.data.DataLoader(trainset, 
                                             batch_size=bsize, 
                                             shuffle=True, 
                                             num_workers=4, 
                                             pin_memory=True, 
                                             drop_last=True)

testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=bsize, 
                                         shuffle=True, 
                                         num_workers=4, 
                                         pin_memory=True, 
                                         drop_last=True)

In [8]:
gamma = 0.4

alpha1 = 10 # triplet
alpha2 = 1e-3 # surjection
alpha3 = 1e-3 # l2 loss


In [9]:
for e in range(n_epochs):
    v_to_s = v_to_s.train()
    s_to_v = s_to_v.train()
    
    for i, sample in enumerate(trainloader):
        optimizer.zero_grad()
        
        batch_visual = sample['image_embedding'].cuda().float()
        batch_semantic = sample['class_embedding'].cuda().float()

        batch_classes = sample['class_id'].cuda()
         
        e_hat = v_to_s(s_to_v(all_class_embeddings, all_class_predicates))
        delta = (e_hat[1] - all_class_embeddings)
        surjection_loss = (delta * delta).sum(dim=-1).mean()
        delta = (e_hat[0] - all_class_predicates)
        surjection_loss = (1-gamma) * surjection_loss + gamma * (delta * delta).sum(dim=-1).mean()
        
        s_out = v_to_s(batch_visual)
        s_attr, s_word = s_out
        
        same_class = classes_enum.unsqueeze(0) == batch_classes.unsqueeze(1)
        same_class = same_class.detach()

        d_matrix = (1 - gamma) * dist_matrix(s_word, all_class_embeddings) + \
                    gamma * dist_matrix(s_attr, all_class_predicates)
        
        ###
        l2_dist_loss = (d_matrix * same_class.float()).mean()
        
        l2_loss = (1-gamma) * (s_word * s_word).sum(dim=-1).mean() + \
                    gamma * (s_attr * s_attr).sum(dim=-1).mean()

        loss = positive_part(furthest_positive - closest_negative + 1)
        loss = alpha1 * loss.mean() + alpha2 * surjection_loss + alpha3 * l2_loss
        
        loss.backward()
        
#         c_hat = d_matrix.argmin(dim = -1)
#         print((c_hat == batch_classes).float().mean().item())
        
        optimizer.step()
        
        print(loss.item(), end=', ')
        
    if (e+1) % 70 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.7
    
    if (e+1) % 5 == 0:
        print('\n\n- Evaluation on epoch {}'.format(e))
        
        avg_accuracy = 0.
        avg_loss = 0.
        n = 0
        
        v_to_s = v_to_s.eval() 
        s_to_v = s_to_v.eval() 
        
        with torch.no_grad():
            for i, sample in enumerate(testloader):
                n += 1
                
                batch_visual = sample['image_embedding'].cuda().float()
                batch_semantic = sample['class_embedding'].cuda().float()

                batch_classes = sample['class_id'].cuda()

                s_out = v_to_s(batch_visual)
                s_attr, s_word = s_out

                same_class = classes_enum.unsqueeze(0) == batch_classes.unsqueeze(1)
                same_class = same_class.detach()

                d_matrix = (1 - gamma) * dist_matrix(s_word, all_class_embeddings) + \
                                gamma * dist_matrix(s_attr, all_class_predicates)
                
                c_hat = d_matrix.argmin(dim = -1)

                closest_negative, _ = (d_matrix + same_class.float() * 1e6).min(dim=-1)
                furthest_positive, _ = (d_matrix * (1 - same_class.float())).max(dim=-1)

                loss = positive_part(furthest_positive - closest_negative + 5)
                loss = alpha1 * furthest_positive.mean()

                avg_loss += loss.item()
                avg_accuracy += (c_hat == batch_classes).float().mean().item()

        avg_accuracy /= n
        avg_loss /= n

        print('Average acc.: {}, Average loss:{}\n\n'.format(avg_accuracy, avg_loss))




10.576552391052246, 10.571242332458496, 10.585657119750977, 10.570113182067871, 10.572693824768066, 10.564399719238281, 10.583442687988281, 10.563982009887695, 10.576849937438965, 10.566195487976074, 10.566062927246094, 10.572005271911621, 10.570043563842773, 10.579184532165527, 10.56896686553955, 10.578373908996582, 10.57557201385498, 10.574932098388672, 10.575716018676758, 10.579255104064941, 10.565694808959961, 10.569279670715332, 10.571930885314941, 10.576912879943848, 10.569809913635254, 10.577178001403809, 10.573882102966309, 10.56824779510498, 10.568777084350586, 10.569540023803711, 10.567770004272461, 10.564340591430664, 10.56863021850586, 10.567337036132812, 10.565375328063965, 10.571569442749023, 10.564568519592285, 10.549568176269531, 10.569628715515137, 10.561238288879395, 10.570425987243652, 10.567354202270508, 10.57348346710205, 10.555363655090332, 10.573796272277832, 10.557287216186523, 10.563777923583984, 10.557394981384277, 10.568008422851562, 10.55777359008789, 10.558

10.473814964294434, 10.472463607788086, 10.471417427062988, 10.47845458984375, 10.471652030944824, 10.477463722229004, 10.476791381835938, 10.465911865234375, 10.474359512329102, 10.47663688659668, 10.476705551147461, 10.47421646118164, 10.475882530212402, 10.474130630493164, 10.475974082946777, 10.466058731079102, 10.470616340637207, 10.476666450500488, 10.47673225402832, 10.468131065368652, 10.47529411315918, 10.473690032958984, 10.471055030822754, 10.477643966674805, 10.472545623779297, 10.476994514465332, 10.471497535705566, 10.475738525390625, 10.475276947021484, 10.477211952209473, 10.472213745117188, 10.477034568786621, 10.467114448547363, 10.470785140991211, 10.467985153198242, 10.469478607177734, 10.46585750579834, 10.468551635742188, 10.46728229522705, 10.474327087402344, 10.469099998474121, 10.47754192352295, 10.475008964538574, 10.472349166870117, 10.467121124267578, 10.469367027282715, 10.469293594360352, 10.469952583312988, 10.474010467529297, 10.463894844055176, 10.47112

10.421088218688965, 10.428439140319824, 10.421414375305176, 10.431675910949707, 10.426183700561523, 10.425251960754395, 10.42251968383789, 10.430203437805176, 10.427746772766113, 10.427931785583496, 10.424474716186523, 10.423203468322754, 10.421954154968262, 10.422307968139648, 10.42325496673584, 10.42492961883545, 10.42291259765625, 10.4251070022583, 10.42514705657959, 10.422247886657715, 10.4263277053833, 10.424420356750488, 10.421612739562988, 10.421426773071289, 10.420987129211426, 10.41860294342041, 10.419031143188477, 10.423933029174805, 10.42479133605957, 10.413914680480957, 10.430465698242188, 10.425638198852539, 10.422989845275879, 10.42755126953125, 10.422779083251953, 10.423589706420898, 10.42320442199707, 10.422731399536133, 10.415956497192383, 10.420620918273926, 10.418737411499023, 10.419844627380371, 10.420681953430176, 10.419843673706055, 10.422870635986328, 10.426002502441406, 10.426702499389648, 10.421138763427734, 10.420893669128418, 10.411648750305176, 10.4199552536

10.389780044555664, 10.388581275939941, 10.392325401306152, 10.385407447814941, 10.385101318359375, 10.387079238891602, 10.39152717590332, 10.383692741394043, 10.385562896728516, 10.387141227722168, 10.392574310302734, 10.3879976272583, 10.389190673828125, 10.384090423583984, 10.38803482055664, 10.386956214904785, 10.381670951843262, 10.386515617370605, 10.384057998657227, 10.385222434997559, 10.391252517700195, 10.39074420928955, 10.388833045959473, 10.38326358795166, 10.388726234436035, 10.382406234741211, 10.386628150939941, 10.3837890625, 10.38699722290039, 10.38428783416748, 10.385071754455566, 10.385074615478516, 10.384552955627441, 10.3930025100708, 10.38403606414795, 10.382936477661133, 10.384008407592773, 10.389664649963379, 10.388554573059082, 10.385189056396484, 10.387979507446289, 10.384016990661621, 10.384309768676758, 10.387726783752441, 10.388214111328125, 10.389104843139648, 10.390302658081055, 10.385220527648926, 10.388657569885254, 10.385799407958984, 10.3816080093383

10.356931686401367, 10.353549003601074, 10.356621742248535, 10.35800552368164, 10.361464500427246, 10.36059284210205, 10.359007835388184, 10.355916023254395, 10.357375144958496, 10.358688354492188, 10.360599517822266, 10.35999870300293, 10.357586860656738, 10.356282234191895, 10.357471466064453, 10.355326652526855, 10.356111526489258, 10.355250358581543, 10.361103057861328, 10.360528945922852, 10.360970497131348, 10.360697746276855, 10.360490798950195, 10.360000610351562, 10.359432220458984, 10.35892391204834, 10.358602523803711, 10.354757308959961, 10.352926254272461, 10.354562759399414, 10.362146377563477, 10.359034538269043, 10.363341331481934, 10.361105918884277, 10.354168891906738, 10.354211807250977, 10.362714767456055, 10.357831001281738, 10.360580444335938, 10.359580993652344, 10.35484790802002, 10.354098320007324, 10.356512069702148, 10.360666275024414, 10.35781478881836, 10.359169960021973, 10.356884956359863, 10.361504554748535, 10.358880043029785, 10.355039596557617, 10.351

...

10.228909492492676, 10.223947525024414, 10.224092483520508, 10.224543571472168, 10.223194122314453, 10.223017692565918, 10.224357604980469, 10.224209785461426, 10.222232818603516, 10.222922325134277, 10.220561981201172, 10.229923248291016, 10.221733093261719, 10.22302532196045, 10.222084045410156, 10.223438262939453, 10.228765487670898, 10.221206665039062, 10.22155475616455, 10.223406791687012, 10.22494888305664, 10.22741985321045, 10.22127628326416, 10.220389366149902, 10.223527908325195, 10.22177505493164, 10.226601600646973, 10.219860076904297, 10.221745491027832, 10.226664543151855, 10.219436645507812, 10.22415542602539, 10.22732925415039, 10.219372749328613, 10.218865394592285, 10.225308418273926, 10.223690032958984, 10.22006607055664, 10.218730926513672, 10.221365928649902, 10.220911979675293, 10.222477912902832, 10.217391014099121, 10.221051216125488, 10.225433349609375, 10.22262954711914, 10.219839096069336, 10.23325252532959, 10.220973014831543, 10.21810245513916, 10.225475311

10.21730899810791, 10.216856956481934, 10.218402862548828, 10.217668533325195, 10.21939754486084, 10.21433162689209, 10.220257759094238, 10.217941284179688, 10.218643188476562, 10.217313766479492, 10.220815658569336, 10.215497970581055, 10.215494155883789, 10.222734451293945, 10.220854759216309, 10.218816757202148, 10.216733932495117, 10.215934753417969, 10.219422340393066, 10.216740608215332, 10.217855453491211, 10.22150707244873, 10.221894264221191, 10.216927528381348, 10.218863487243652, 10.21996784210205, 10.215937614440918, 10.2210111618042, 10.21967601776123, 10.215180397033691, 10.218958854675293, 10.218128204345703, 10.217766761779785, 10.219337463378906, 10.217061042785645, 10.217667579650879, 10.221380233764648, 10.215574264526367, 10.216870307922363, 10.216899871826172, 10.213994979858398, 10.215292930603027, 10.21312427520752, 10.22116756439209, 10.218613624572754, 10.219717025756836, 10.219171524047852, 10.220040321350098, 10.219868659973145, 10.218411445617676, 10.2163238

10.217594146728516, 10.210453987121582, 10.209402084350586, 10.21205997467041, 10.214550018310547, 10.210954666137695, 10.213065147399902, 10.215622901916504, 10.211838722229004, 10.213516235351562, 10.217329978942871, 10.217350959777832, 10.214166641235352, 10.212203025817871, 10.211584091186523, 10.209515571594238, 10.212644577026367, 10.210111618041992, 10.214520454406738, 10.21146297454834, 10.216190338134766, 10.214681625366211, 10.213846206665039, 10.214966773986816, 10.214344024658203, 10.216261863708496, 10.216092109680176, 10.215261459350586, 10.210403442382812, 10.211742401123047, 10.21306037902832, 10.217960357666016, 10.217479705810547, 10.215717315673828, 10.210357666015625, 10.208675384521484, 10.219587326049805, 10.213189125061035, 10.214136123657227, 10.210221290588379, 10.2109375, 10.215510368347168, 10.207810401916504, 10.214205741882324, 10.215084075927734, 10.210325241088867, 10.20970344543457, 10.214872360229492, 10.210503578186035, 10.216033935546875, 10.216620445

10.207953453063965, 10.207883834838867, 10.207527160644531, 10.208725929260254, 10.207771301269531, 10.206258773803711, 10.208145141601562, 10.208159446716309, 10.207987785339355, 10.209508895874023, 10.20935344696045, 10.206942558288574, 10.206900596618652, 10.209486961364746, 10.205013275146484, 10.20931339263916, 10.207016944885254, 10.210288047790527, 10.207576751708984, 10.210003852844238, 10.208322525024414, 10.20926284790039, 10.206392288208008, 10.207622528076172, 10.207440376281738, 10.213780403137207, 10.20853328704834, 10.20726203918457, 10.207549095153809, 10.204448699951172, 10.20775318145752, 10.21550178527832, 10.209959983825684, 10.204283714294434, 10.20522689819336, 10.209728240966797, 10.2131986618042, 10.206775665283203, 10.207565307617188, 10.210044860839844, 10.21066665649414, 10.206098556518555, 10.209321022033691, 10.209357261657715, 10.21132755279541, 10.205766677856445, 10.207047462463379, 10.205667495727539, 10.205832481384277, 10.212810516357422, 10.212116241

10.205192565917969, 10.207000732421875, 10.202558517456055, 10.208416938781738, 10.205802917480469, 10.201812744140625, 10.202725410461426, 10.198649406433105, 10.203556060791016, 10.20697021484375, 10.203592300415039, 10.204753875732422, 10.2039213180542, 10.204141616821289, 10.204055786132812, 10.204879760742188, 10.205310821533203, 10.20627212524414, 10.207159042358398, 10.20808219909668, 10.200845718383789, 10.207062721252441, 10.203288078308105, 10.204103469848633, 10.20678424835205, 10.202285766601562, 10.20561408996582, 10.202993392944336, 10.202098846435547, 10.2027587890625, 10.201300621032715, 10.204837799072266, 10.201014518737793, 10.202437400817871, 10.208267211914062, 10.20390796661377, 10.203934669494629, 10.20421028137207, 10.202451705932617, 10.203739166259766, 10.206840515136719, 10.206335067749023, 10.209610939025879, 10.203271865844727, 10.197952270507812, 10.197551727294922, 10.203694343566895, 10.203010559082031, 10.204320907592773, 10.19848918914795, 10.199051856